In [1]:
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast
import warnings
warnings.filterwarnings("ignore")

In [2]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [3]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\qyxx-mv3-KA.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\qyxx-mv3-KA.pkl"  #修改此处文件名

In [4]:
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [5]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [6]:
from torchvision import models

mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
mobilenet_v3_large

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

In [7]:
class ECAAttention(nn.Module):

    def __init__(self, kernel_size=7):
        super().__init__()
        self.gap=nn.AdaptiveAvgPool2d(1)
        self.maxpool=nn.AdaptiveMaxPool2d(1)
        self.conv=nn.Conv1d(1,1,kernel_size=kernel_size,padding=3)
        self.sigmoid=nn.Sigmoid()
        self.init_weights()
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        avp_result =self.gap(x) #bs,c,1,1
        max_result = self.maxpool(x)
        
        avp_result=avp_result.squeeze(-1).permute(0,2,1) #bs,1,c
        max_result=max_result.squeeze(-1).permute(0,2,1) #bs,1,c
        
        avp_result=self.conv(avp_result) #bs,1,c
        max_result=self.conv(max_result) #bs,1,c
        
        y=self.sigmoid(max_result + avp_result) #bs,1,c
        y=y.permute(0,2,1).unsqueeze(-1) #bs,c,1,1
        return x*y.expand_as(x)

class SpatialAttention(nn.Module):
    def __init__(self,kernel_size=7):
        super().__init__()
        self.conv=nn.Conv2d(2,1,kernel_size=kernel_size,padding=3)
        self.sigmoid=nn.Sigmoid()
        self.init_weights()
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
    def forward(self, x) :
        max_result,_=torch.max(x,dim=1,keepdim=True)
        avg_result=torch.mean(x,dim=1,keepdim=True)
        result=torch.cat([max_result,avg_result],1)
        output=self.conv(result)
        output=self.sigmoid(output)
        return x*output
    
class CA_SA(nn.Module):
    def __init__(self):
        super().__init__()
        self.eca = ECAAttention()
        self.sa = SpatialAttention()
        
    def forward(self,x):
        x = self.eca(x)
        x = self.sa(x)
        return x
    
class K_Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.eca = ECAAttention()
        self.sa = SpatialAttention()
        self.eca_sa = CA_SA()
    def forward(self,x):
        out1 = self.eca_sa(x)
        out2 = self.eca(x) + self.sa(x)
        return out2 + out1

In [8]:
class SKNet(nn.Module):
    def __init__(self, num_class=7):
        super(SKNet, self).__init__()
        self.features = nn.Sequential(*list(mobilenet_v3_large.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.KA = K_Attention()
        self.fc = nn.Linear(960*4, num_class)

    def forward(self, x):
        
        x = self.features(x)
        out1 = self.KA(x)
        out2 = self.KA(x)
        out3 = self.KA(x)
        out4 = self.KA(x)
        out = torch.cat((out1,out2,out3,out4),dim=1)
        out = self.avgpool(out)
        out = torch.flatten(out,1)
        out = self.fc(out)
    
        return out

In [9]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [10]:
#to(device)将模型加入GPU中加速计算
model = SKNet().to(device)
#设置优化器
params = list(model.parameters()) 
optimizer = optim.AdamW(params, lr=lr)

# optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 16, 112, 112]             432
       BatchNorm2d-2        [128, 16, 112, 112]              32
         Hardswish-3        [128, 16, 112, 112]               0
            Conv2d-4        [128, 16, 112, 112]             144
       BatchNorm2d-5        [128, 16, 112, 112]              32
              ReLU-6        [128, 16, 112, 112]               0
            Conv2d-7        [128, 16, 112, 112]             256
       BatchNorm2d-8        [128, 16, 112, 112]              32
  InvertedResidual-9        [128, 16, 112, 112]               0
           Conv2d-10        [128, 64, 112, 112]           1,024
      BatchNorm2d-11        [128, 64, 112, 112]             128
             ReLU-12        [128, 64, 112, 112]               0
           Conv2d-13          [128, 64, 56, 56]             576
      BatchNorm2d-14          [128, 64,

In [11]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [12]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label)  
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 38.90 s
Accuracy : Train is 0.6434210526315789 , Valid is 0.6266983695652174 ;  Loss : Train is  0.9920649177149723 ,Valid is 1.108384062414584
覆盖最好的模型...
epoch: 1
The Model-Train-Time spent  0 min 33.47 s
Accuracy : Train is 0.8033717105263157 , Valid is 0.7639266304347826 ;  Loss : Train is  0.5516395151615143 ,Valid is 0.7199715220409891
覆盖最好的模型...
epoch: 2
The Model-Train-Time spent  0 min 33.02 s
Accuracy : Train is 0.8698190789473684 , Valid is 0.7924592391304348 ;  Loss : Train is  0.37246502449637964 ,Valid is 0.61358057545579
覆盖最好的模型...
epoch: 3
The Model-Train-Time spent  0 min 32.92 s
Accuracy : Train is 0.9180921052631579 , Valid is 0.7829483695652174 ;  Loss : Train is  0.24275878087470407 ,Valid is 0.7166607690894086
epoch: 4
The Model-Train-Time spent  0 min 32.86 s
Accuracy : Train is 0.9405427631578948 , Valid is 0.7822690217391304 ;  Loss : Train is  0.16486461844883468 ,Valid is 0.8084870343622954
epoch: 5
The Model-Train-Ti

The Model-Train-Time spent  0 min 32.87 s
Accuracy : Train is 0.9977796052631579 , Valid is 0.8325407608695652 ;  Loss : Train is  0.008028297626266353 ,Valid is 2.6943787336349487
epoch: 44
The Model-Train-Time spent  0 min 32.77 s
Accuracy : Train is 0.9983552631578947 , Valid is 0.8468070652173912 ;  Loss : Train is  0.009542430054984594 ,Valid is 2.35853824408158
覆盖最好的模型...
epoch: 45
The Model-Train-Time spent  0 min 32.74 s
Accuracy : Train is 0.9990131578947368 , Valid is 0.8342391304347826 ;  Loss : Train is  0.004352533934932006 ,Valid is 2.6236756469892417
epoch: 46
The Model-Train-Time spent  0 min 32.81 s
Accuracy : Train is 0.9989309210526316 , Valid is 0.8383152173913043 ;  Loss : Train is  0.006228383140344369 ,Valid is 2.8195038930229517
epoch: 47
The Model-Train-Time spent  0 min 33.63 s
Accuracy : Train is 0.9994243421052631 , Valid is 0.8403532608695652 ;  Loss : Train is  0.0014123746046894475 ,Valid is 2.8036995037742285
epoch: 48
The Model-Train-Time spent  0 min 3

Accuracy : Train is 1.0 , Valid is 0.8536005434782609 ;  Loss : Train is  9.313225746154785e-10 ,Valid is 2.899934929350148
epoch: 89
The Model-Train-Time spent  0 min 32.86 s
Accuracy : Train is 1.0 , Valid is 0.8519021739130435 ;  Loss : Train is  1.1685647462543688e-08 ,Valid is 2.920217047566953
epoch: 90
The Model-Train-Time spent  0 min 32.84 s
Accuracy : Train is 1.0 , Valid is 0.8522418478260869 ;  Loss : Train is  8.823055970041376e-11 ,Valid is 2.899400856183923
epoch: 91
The Model-Train-Time spent  0 min 33.48 s
Accuracy : Train is 1.0 , Valid is 0.8539402173913043 ;  Loss : Train is  3.085128570857801e-08 ,Valid is 2.8638048327487446
epoch: 92
The Model-Train-Time spent  0 min 33.45 s
Accuracy : Train is 1.0 , Valid is 0.8563179347826086 ;  Loss : Train is  5.225209813368948e-09 ,Valid is 2.9049146615940593
覆盖最好的模型...
epoch: 93
The Model-Train-Time spent  0 min 32.70 s
Accuracy : Train is 1.0 , Valid is 0.8522418478260869 ;  Loss : Train is  1.4705093283402293e-09 ,Valid is

The Model-Train-Time spent  0 min 32.81 s
Accuracy : Train is 0.9921875 , Valid is 0.819633152173913 ;  Loss : Train is  0.026316952607349346 ,Valid is 1.4636506930641506
epoch: 135
The Model-Train-Time spent  0 min 32.84 s
Accuracy : Train is 0.9912828947368421 , Valid is 0.8247282608695652 ;  Loss : Train is  0.030233303947668325 ,Valid is 1.5483371589494788
epoch: 136
The Model-Train-Time spent  0 min 32.85 s
Accuracy : Train is 0.9926809210526315 , Valid is 0.8277853260869565 ;  Loss : Train is  0.024538034837889043 ,Valid is 1.4763813226119331
epoch: 137
The Model-Train-Time spent  0 min 32.87 s
Accuracy : Train is 0.9930921052631578 , Valid is 0.8209918478260869 ;  Loss : Train is  0.023721953520649357 ,Valid is 1.6023720710173897
epoch: 138
The Model-Train-Time spent  0 min 32.78 s
Accuracy : Train is 0.9930921052631578 , Valid is 0.8376358695652174 ;  Loss : Train is  0.027223889441474487 ,Valid is 1.4197625414184902
epoch: 139
The Model-Train-Time spent  0 min 32.82 s
Accuracy

Accuracy : Train is 0.9941611842105262 , Valid is 0.8250679347826086 ;  Loss : Train is  0.02677744292703114 ,Valid is 2.2856321593989497
epoch: 178
The Model-Train-Time spent  0 min 32.87 s
Accuracy : Train is 0.9959703947368421 , Valid is 0.8325407608695652 ;  Loss : Train is  0.017851336065091584 ,Valid is 2.330482332602791
epoch: 179
The Model-Train-Time spent  0 min 32.80 s
Accuracy : Train is 0.9951480263157895 , Valid is 0.8322010869565217 ;  Loss : Train is  0.030203706535853837 ,Valid is 2.4879572598830513
epoch: 180
The Model-Train-Time spent  0 min 33.61 s
Accuracy : Train is 0.9967927631578947 , Valid is 0.8308423913043478 ;  Loss : Train is  0.012398600029317957 ,Valid is 2.492795793906502
epoch: 181
The Model-Train-Time spent  0 min 34.30 s
Accuracy : Train is 0.9967927631578947 , Valid is 0.8301630434782609 ;  Loss : Train is  0.0174961881810113 ,Valid is 2.9711790240329243
epoch: 182
The Model-Train-Time spent  0 min 33.46 s
Accuracy : Train is 0.9980263157894737 , Vali